In [1]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from itertools import combinations
from collections import defaultdict

/Users/JakeCanfield/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
data_fp = '/Users/JakeCanfield/Documents/Trauma_Surgery_Research/Data/Raw_data/PUF AY %i/CSV/'

mechanism_code_dict = {1:'Cut/pierce', 2:'Drowning/submersion', 3:'Fall', 4:'Fire/flame', 5:'Hot object/substance', 6:'Firearm', 7:'Machinery', 8:'MVT Occupant', 9:'MVT Motorcyclist', 10:'MVT Pedal cyclist', 11:'MVT Pedestrian', 12:'MVT Unspecified', 13:'MVT Other', 14:'Pedal cyclist, other', 15:'Pedestrian, other', 16:'Transport, other', 17:'Natural/environmental,  Bites and stings', 18:'Natural/environmental,  Other', 19:'Overexertion', 20:'Poisoning', 21:'Struck by, against', 22:'Suffocation', 23:'Other specified and classifiable', 24:'Other specified, not elsewhere classifiable', 25:'Unspecified', 26:'Adverse effects, medical care', 27:'Adverse effects, drugs'} # As noted in PUF dictionary
trauma_type_code_dict = {1:'Blunt', 2:'Penetrating', 3:'Burn', 4:'Other/unspecified', 9:'Activity Code - Not Valid as a Primary E-Code'} # As noted in PUF Dictionary
sex_code_dict = {1:'Male', 2:'Female', 3:'Unknown'}
eddischarge_code_dict = {1: 'Floor bed (general admission, non-specialty unit bed)', 2: 'Observation unit (unit that provides < 24 hour stays)', 3: 'Telemetry/step-down unit (less acuity than ICU)', 4: 'Home with services', 5: 'Deceased/expired', 6: 'Other (jail, institutional care, mental health, etc.)', 7: 'Operating Room', 8: 'Intensive Care Unit (ICU)', 9: 'Home without services', 10: 'Left against medical advice', 11: 'Transferred to another hospital'}
hospdischarge_disposition_code_dict = {1: 'Discharged/Transferred to a short-term general hospital for inpatient care', 2: 'Discharged/Transferred to an Intermediate Care Facility (ICF)', 3: 'Discharged/Transferred to home under care of organized home health service', 4: 'Left against medical advice or discontinued care', 5: 'Deceased/Expired', 6: 'Discharged to home or self-care (routine discharge)', 7: 'Discharged/Transferred to Skilled Nursing Facility (SNF)', 8: 'Discharged/Transferred to hospice care', 10: 'Discharged/Transferred to court/law enforcement', 11: 'Discharged/Transferred to inpatient rehab or designated unit', 12: 'Discharged/Transferred to Long Term Care Hospital (LTCH)', 13: 'Discharged/Transferred to a psychiatric hospital or psychiatric distinct part unit of a hospital', 14: 'Discharged/Transferred to another type of institution not defined elsewhere'}
prehospca_code_dict = {1:'Yes', 2:'No', 3:'Unknown'}

vitals_cols = ['EMSSBP', 'EMSRESPIRATORYRATE', 'EMSPULSERATE', 'EMSPULSEOXIMETRY', 'EMSTOTALGCS',
              'TEMPERATURE', 'SBP', 'RESPIRATORYRATE', 'PULSERATE', 'PULSEOXIMETRY', 'TOTALGCS']
required_cols = ['INC_KEY', 'SEX', 'TRAUMATYPE', 'MECHANISM', 'AGEYEARS', 'EDDISCHARGEDISPOSITION', 'HOSPDISCHARGEDISPOSITION', 'PREHOSPITALCARDIACARREST', 'HMRRHGCTRLSURGMINS']

In [4]:
years = range(2017, 2021, 1)
cutofftimes = [20]
for cutofftime in cutofftimes:
    for year in years:
        if year in range(2017, 2019):
            TRAUMA_df = pd.read_csv(data_fp%year + 'PUF_TRAUMA.csv')
            ECODE_LOOKUP_df = pd.read_csv(data_fp%year + 'PUF_ECODE_LOOKUP.csv')
        elif year in range(2019, 2021):
            TRAUMA_df = pd.read_csv(data_fp%year + 'PUF_TRAUMA.csv')
            ECODE_LOOKUP_df = pd.read_csv(data_fp%year + 'PUF_ECODE_LOOKUP.csv')
        else:
            pass
        
        TRAUMA_df.columns = map(str.upper, TRAUMA_df.columns)
        ECODE_LOOKUP_df.columns = map(str.upper, ECODE_LOOKUP_df.columns)
        
        if year in range(2019, 2021):
            TRAUMA_df['HMRRHGCTRLSURGMINS'] = TRAUMA_df['HMRRHGCTRLSURGHRS']*60
            TRAUMA_df['EDMINS'] = TRAUMA_df['EDDISCHARGEHRS']*60
        else:
            pass
        
        TRAUMA_df = TRAUMA_df.dropna(subset=['HMRRHGCTRLSURGMINS'])
        TRAUMA_df['HMRRHGCTRLSURGMINS'] = TRAUMA_df['HMRRHGCTRLSURGMINS'].round().astype(int)
        
        TRAUMA_df = TRAUMA_df.dropna(subset=['EDMINS'])
        TRAUMA_df['EDMINS'] = TRAUMA_df['EDMINS'].round().astype(int)
        
        TRAUMA_df = TRAUMA_df.loc[TRAUMA_df['HMRRHGCTRLSURGTYPE'] == 3.0]
        
        TRAUMA_df = TRAUMA_df[(TRAUMA_df['HMRRHGCTRLSURGMINS'] <= TRAUMA_df['EDMINS']) & (TRAUMA_df['HMRRHGCTRLSURGMINS'] <= cutofftime)]
        
        ECODE_LOOKUP_df = ECODE_LOOKUP_df.loc[:, ['ECODE', 'ECODE_DESC', 'MECHANISM', 'TRAUMATYPE']]
    
        ecode_dict = dict(zip(ECODE_LOOKUP_df['ECODE'], ECODE_LOOKUP_df['ECODE_DESC'])) # create dictionary
        TRAUMA_df['PRIMARYECODEICD10'] = TRAUMA_df['PRIMARYECODEICD10'].replace(ecode_dict) # implement dictionary
        
        mechanism_dict = dict(zip(ECODE_LOOKUP_df['ECODE_DESC'], ECODE_LOOKUP_df['MECHANISM'])) # create dictionary
        TRAUMA_df['MECHANISM'] = TRAUMA_df['PRIMARYECODEICD10'].map(mechanism_dict) # implement dictionary
        
        traumatype_dict = dict(zip(ECODE_LOOKUP_df['ECODE_DESC'], ECODE_LOOKUP_df['TRAUMATYPE'])) # create dictionary
        TRAUMA_df['TRAUMATYPE'] = TRAUMA_df['PRIMARYECODEICD10'].map(traumatype_dict) # implement dictionary
        
        TRAUMA_df['MECHANISM'] = TRAUMA_df['MECHANISM'].replace(mechanism_code_dict) # implement dictionary
        
        TRAUMA_df['TRAUMATYPE'] = TRAUMA_df['TRAUMATYPE'].replace(trauma_type_code_dict) # implement dictionary
        
        TRAUMA_df['SEX'] = TRAUMA_df['SEX'].replace(sex_code_dict)
        
        TRAUMA_df['EDDISCHARGEDISPOSITION'] = TRAUMA_df['EDDISCHARGEDISPOSITION'].replace(eddischarge_code_dict) # implement dictionary
    
        TRAUMA_df['HOSPDISCHARGEDISPOSITION'] = TRAUMA_df['HOSPDISCHARGEDISPOSITION'].replace(hospdischarge_disposition_code_dict) # implement dictionary
        
        TRAUMA_df['PREHOSPITALCARDIACARREST'] = TRAUMA_df['PREHOSPITALCARDIACARREST'].apply(lambda x: x if x in [1, 2] else 3)
        TRAUMA_df['PREHOSPITALCARDIACARREST'] = TRAUMA_df['PREHOSPITALCARDIACARREST'].replace(prehospca_code_dict) # implement dictionary
        
        TRAUMA_df = TRAUMA_df.loc[:,required_cols+vitals_cols]
        numeric_columns = TRAUMA_df.select_dtypes(include=[np.number]).columns
        TRAUMA_df[numeric_columns] = TRAUMA_df[numeric_columns].applymap(lambda x: np.nan if x < 0 else x)
        print('Number of Cases in %i (Cutoff after arrival = %i minutes):'%(year, cutofftime), np.shape(TRAUMA_df)[0])
        
        TRAUMA_df.to_csv('/Users/JakeCanfield/Documents/Trauma_Surgery_Research/data/Vitals/vitals_%i_mins_df_%i.csv'%(cutofftime, year), index=False)

Number of Cases in 2017 (Cutoff after arrival = 20 minutes): 859
Number of Cases in 2018 (Cutoff after arrival = 20 minutes): 880
Number of Cases in 2019 (Cutoff after arrival = 20 minutes): 1081


/Users/JakeCanfield/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3325: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Number of Cases in 2020 (Cutoff after arrival = 20 minutes): 1385


In [24]:
import pandas as pd
import numpy as np

years = range(2007, 2017)
cutofftimes = [20]
data_fp = '/Users/JakeCanfield/Documents/Trauma_Surgery_Research/Data/Raw_data/PUF AY %i/CSV/'  # Define your data file path here

for cutofftime in cutofftimes:
    for year in years:
        PCODE_df = pd.read_csv(data_fp % year + 'PUF_PCODE.csv').sort_values('INC_KEY')
        if year in range(2010, 2017):
            PCODE_df = PCODE_df.rename(columns={'HOURTOPROC': 'HOURTOPRO'})
        PCODEDES_df = pd.read_csv(data_fp % year + 'PUF_PCODEDES.csv')
        ECODE_df = pd.read_csv(data_fp % year + 'PUF_ECODE.csv', usecols=['INC_KEY', 'ECODE']).sort_values('INC_KEY')
        ECODEDES_df = pd.read_csv(data_fp % year + 'PUF_ECODEDES.csv', usecols=['INJTYPE', 'MECHANISM', 'ECODE', 'ECODEDES'])
        DEMO_df = pd.read_csv(data_fp % year + 'PUF_DEMO.csv', usecols=['INC_KEY', 'GENDER', 'AGE']).sort_values('INC_KEY')
        DISCHARGE_df = pd.read_csv(data_fp % year + 'PUF_DISCHARGE.csv', usecols=['INC_KEY', 'HOSPDISP']).sort_values('INC_KEY')
        VITALS_df = pd.read_csv(data_fp % year + 'PUF_VITALS.csv', usecols=['INC_KEY', 'VSTYPE', 'SBP', 'RR', 'PULSE', 'OXYSAT', 'TEMP', 'GCSTOT']).sort_values('INC_KEY')
        ED_df = pd.read_csv(data_fp % year + 'PUF_ED.csv', usecols=['INC_KEY', 'EDMIN', 'EDDISP']).sort_values('INC_KEY')
        if year in range(2013, 2017):
            PM_df = pd.read_csv(data_fp % year + 'PUF_PM.csv', usecols=['inc_key', 'HEMORRHAGE_CTRL_STYPE_CODE', 'HEMORRHAGE_CTRL_STYPE_DESC', 'HemorrhageCtrlMins', 'HemorrhageCtrlDays']).sort_values('inc_key')
            PM_df.columns = map(str.upper, PM_df.columns)
            PM_df = pd.merge(PM_df, ED_df[['INC_KEY', 'EDMIN']], on='INC_KEY', how='left')
            PM_df = PM_df.dropna(subset=['HEMORRHAGECTRLMINS'])
            PM_df['HEMORRHAGECTRLMINS'] = PM_df['HEMORRHAGECTRLMINS'].round().astype(int)
        else:
            PM_df = pd.DataFrame()  # Empty DataFrame for consistency
        
        # Force column headers to be all upper case and fill in missing year data
        PCODE_df.columns = map(str.upper, PCODE_df.columns)
        ED_df.columns = map(str.upper, ED_df.columns)
        PCODEDES_df.columns = map(str.upper, PCODEDES_df.columns)
        PCODE_df['YOPROC'] = PCODE_df['YOPROC'].fillna(year)
        
        PCODE_df['PROCEDUREMINS'] = PCODE_df['HOURTOPRO'] * 60
        PCODE_df = PCODE_df.dropna(subset=['PROCEDUREMINS'])
        PCODE_df['PROCEDUREMINS'] = PCODE_df['PROCEDUREMINS'].round().astype(int)
        
        PCODE_df = PCODE_df.loc[:, ['INC_KEY', 'PCODE', 'DAYTOPROC', 'PROCEDUREMINS', 'HOURTOPRO']]
        
        ECODE_LOOKUP_df = ECODEDES_df.loc[:, ['ECODE', 'ECODEDES', 'MECHANISM', 'INJTYPE']]
        
        # Merge ED_df with PCODE_df to add EDMIN column
        PCODE_df = pd.merge(PCODE_df, ED_df[['INC_KEY', 'EDMIN']], on='INC_KEY', how='left')
    
        if year in range(2013, 2017):
            DCS_list = PM_df.loc[PM_df['HEMORRHAGE_CTRL_STYPE_CODE'] == 3.0]
            DCS_list = DCS_list[DCS_list['HEMORRHAGECTRLMINS'] <= cutofftime]
            thoracotomy_list = DCS_list[DCS_list['HEMORRHAGECTRLMINS'] <= DCS_list['EDMIN']]
            thoracotomy_list2 = PCODE_df.loc[PCODE_df['PCODE'] == 34.02]
            thoracotomy_list2 = thoracotomy_list2[(thoracotomy_list2['PROCEDUREMINS'] <= cutofftime) & (thoracotomy_list2['PROCEDUREMINS'] <= thoracotomy_list2['EDMIN'])]
            thoracotomy_keys = np.unique(thoracotomy_list['INC_KEY'].tolist() + thoracotomy_list2['INC_KEY'].tolist())
        else:
            thoracotomy_list = PCODE_df.loc[PCODE_df['PCODE'] == 34.02]
            thoracotomy_list = thoracotomy_list[(thoracotomy_list['PROCEDUREMINS'] <= cutofftime) & (thoracotomy_list['PROCEDUREMINS'] <= thoracotomy_list['EDMIN'])]
            thoracotomy_keys = np.unique(thoracotomy_list['INC_KEY'].tolist())
        PCODE_df = PCODE_df.loc[(PCODE_df['PCODE'] == 34.02) & (PCODE_df['PROCEDUREMINS'] <= cutofftime) & (PCODE_df['PROCEDUREMINS'] <= PCODE_df['EDMIN'])]
        PCODE_df = PCODE_df.loc[PCODE_df['INC_KEY'].isin(thoracotomy_keys)]
        ECODE_df = ECODE_df.loc[ECODE_df['INC_KEY'].isin(thoracotomy_keys)]
        DEMO_df = DEMO_df.loc[DEMO_df['INC_KEY'].isin(thoracotomy_keys)]
        DISCHARGE_df = DISCHARGE_df.loc[DISCHARGE_df['INC_KEY'].isin(thoracotomy_keys)]
        ED_df = ED_df.loc[ED_df['INC_KEY'].isin(thoracotomy_keys)]
        VITALS_df = VITALS_df.loc[VITALS_df['INC_KEY'].isin(thoracotomy_keys)]
        if year in range(2013, 2017):
            PM_df = PM_df.loc[PM_df['INC_KEY'].isin(thoracotomy_keys)]
        
        ecode_dict = dict(zip(ECODEDES_df['ECODE'], ECODEDES_df['ECODEDES']))
        ECODE_df['ECODE'] = ECODE_df['ECODE'].replace(ecode_dict)
        
        mechanism_dict = dict(zip(ECODEDES_df['ECODEDES'], ECODEDES_df['MECHANISM']))
        ECODE_df['MECHANISM'] = ECODE_df['ECODE'].map(mechanism_dict)
        
        traumatype_dict = dict(zip(ECODEDES_df['ECODEDES'], ECODE_LOOKUP_df['INJTYPE']))
        ECODE_df['INJTYPE'] = ECODE_df['ECODE'].map(traumatype_dict)
        
        DEMO_df['GENDER'] = DEMO_df['GENDER'].replace(sex_code_dict)
        ED_df['EDDISP'] = ED_df['EDDISP'].replace(eddischarge_code_dict)
        DISCHARGE_df['HOSPDISP'] = DISCHARGE_df['HOSPDISP'].replace(hospdischarge_disposition_code_dict)
        ED_df['PREHOSPITALCARDIACARREST'] = 3
        
        # Pivot VITALS_df
        VITALS_pivot = VITALS_df.pivot(index='INC_KEY', columns='VSTYPE', values=['SBP', 'RR', 'PULSE', 'OXYSAT', 'TEMP', 'GCSTOT'])
        VITALS_pivot.columns = [f'{col[1]}{col[0]}' for col in VITALS_pivot.columns]
        VITALS_pivot = VITALS_pivot.reset_index()   
        
        if year in range(2013, 2017):
            TRAUMA_df = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(ECODE_df, DEMO_df, on='INC_KEY', how='left'), ED_df, on='INC_KEY', how='left'), PM_df, on='INC_KEY', how='left'), VITALS_pivot, on='INC_KEY', how='left'), DISCHARGE_df, on='INC_KEY', how='left'), PCODE_df, on='INC_KEY', how='left')
            TRAUMA_df['HMRRHGCTRLSURGMINS'] = TRAUMA_df['HEMORRHAGECTRLMINS'].fillna(TRAUMA_df['PROCEDUREMINS'])
        else:
            TRAUMA_df = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(ECODE_df, DEMO_df, on='INC_KEY', how='left'), ED_df, on='INC_KEY', how='left'), VITALS_pivot, on='INC_KEY', how='left'), DISCHARGE_df, on='INC_KEY', how='left'), PCODE_df, on='INC_KEY', how='left')
            TRAUMA_df['HMRRHGCTRLSURGMINS'] = TRAUMA_df['PROCEDUREMINS']
        
        TRAUMA_df = TRAUMA_df.rename(columns={'INJTYPE':'TRAUMATYPE', 'GENDER':'SEX', 'AGE':'AGEYEARS',
                                             'EDSBP':'SBP', 'EDRR':'RESPIRATORYRATE', 'EMSRR':'EMSRESPIRATORYRATE',
                                             'EDPULSE':'PULSERATE', 'EMSPULSE':'EMSPULSERATE', 'EDOXYSAT':'PULSEOXIMETRY',
                                             'EMSOXYSAT':'EMSPULSEOXIMETRY', 'EDTEMP':'TEMPERATURE',
                                             'EDGCSTOT':'TOTALGCS', 'EMSGCSTOT':'EMSTOTALGCS', 
                                             'EDDISP':'EDDISCHARGEDISPOSITION', 'HOSPDISP':'HOSPDISCHARGEDISPOSITION'})
        
        TRAUMA_df = TRAUMA_df.loc[:, required_cols + vitals_cols + ['HMRRHGCTRLSURGMINS']]
        TRAUMA_df = TRAUMA_df.drop_duplicates(subset=['INC_KEY'])
        
        numeric_columns = TRAUMA_df.select_dtypes(include=[np.number]).columns
        TRAUMA_df[numeric_columns] = TRAUMA_df[numeric_columns].applymap(lambda x: np.nan if x < 0 else x)
        
        print('Number of Cases in %i (Cutoff after arrival = %i minutes):' % (year, cutofftime), np.shape(TRAUMA_df)[0])
        print(np.shape(TRAUMA_df))
        print(np.shape(np.unique(TRAUMA_df['INC_KEY'])))
        display(np.unique(TRAUMA_df['EDDISCHARGEDISPOSITION']))
        display(np.unique(TRAUMA_df['HOSPDISCHARGEDISPOSITION']))
        
        TRAUMA_df.to_csv('/Users/JakeCanfield/Documents/Trauma_Surgery_Research/data/Vitals/vitals_%i_mins_df_%i.csv' % (cutofftime, year), index=False)


Number of Cases in 2007 (Cutoff after arrival = 20 minutes): 400
(400, 20)
(400,)


array(['Died', 'Floor bed (general admission, non specialty unit bed)',
       'Intensive Care Unit (ICU)', 'Not Applicable BIU 1',
       'Not Known BIU 2', 'Not Recorded BIU 3', 'Operating Room',
       'Other (jail, institutional care facility, mental health, etc)',
       'Telemetry/step-down unit (less acuity than ICU)'], dtype=object)

array(['Discharge/Transferred to home under care of Home Health Agency',
       'Discharged home with no home services',
       'Discharged/Transferred to Skilled Nursing Facility',
       'Discharged/Transferred to an Intermediate Care Facility',
       'Discharged/Transferred to another acute care hospital using EMS',
       'Discharged/Transferred to another type of rehabilitation or long term',
       'Expired', 'Left against medical advice', 'Not Applicable BIU 1',
       'Not Known BIU 2', 'Not Recorded BIU 3'], dtype=object)

Number of Cases in 2008 (Cutoff after arrival = 20 minutes): 912
(912, 20)
(912,)


array(['Died', 'Floor bed (general admission, non specialty unit bed)',
       'Home without services', 'Intensive Care Unit (ICU)',
       'Not Applicable BIU 1', 'Not Known BIU 2', 'Operating Room',
       'Telemetry/step-down unit (less acuity than ICU)',
       'Transferred to another hospital'], dtype=object)

...

array(['Deceased/Expired',
       'Floor bed (general admission, non specialty unit bed)',
       'Home with services', 'Home without services',
       'Intensive Care Unit (ICU)', 'Not Known/Not Recorded BIU 2',
       'Operating Room',
       'Telemetry/step-down unit (less acuity than ICU)'], dtype=object)

array(['Deceased/Expired',
       'Discharge/Transferred to home under care of organized home health serv',
       'Discharged to home or self-care (routine discharge)',
       'Discharged/Transferred to Long Term Care Hospital',
       'Discharged/Transferred to Skilled Nursing Facility',
       'Discharged/Transferred to a short-term general hospital for inpatient',
       'Discharged/Transferred to an Intermediate Care Facility (ICF)',
       'Discharged/Transferred to another type of institution not defined else',
       'Discharged/Transferred to court/law enforcement',
       'Discharged/Transferred to inpatient rehab or designated unit',
       'Discharged/transferred to a psychiatric hospital or psychiatric distin',
       'Left against medical advice or discontinued care',
       'Not Applicable BIU 1', 'Not Known/Not Recorded BIU 2'],
      dtype=object)

Number of Cases in 2016 (Cutoff after arrival = 20 minutes): 855
(855, 20)
(855,)


array(['Deceased/Expired',
       'Floor bed (general admission, non specialty unit bed)',
       'Home with services', 'Home without services',
       'Intensive Care Unit (ICU)', 'Not Applicable BIU 1',
       'Operating Room',
       'Other (jail, institutional care facility, mental health, etc)',
       'Telemetry/step-down unit (less acuity than ICU)',
       'Transferred to another hospital'], dtype=object)

array(['Deceased/Expired',
       'Discharge/Transferred to home under care of organized home health serv',
       'Discharged to home or self-care (routine discharge)',
       'Discharged/Transferred to Long Term Care Hospital',
       'Discharged/Transferred to Skilled Nursing Facility',
       'Discharged/Transferred to a short-term general hospital for inpatient',
       'Discharged/Transferred to another type of institution not defined else',
       'Discharged/Transferred to court/law enforcement',
       'Discharged/Transferred to inpatient rehab or designated unit',
       'Discharged/transferred to a psychiatric hospital or psychiatric distin',
       'Left against medical advice or discontinued care',
       'Not Applicable BIU 1'], dtype=object)

In [23]:
display(np.unique(TRAUMA_df['HOSPDISCHARGEDISPOSITION']))

array(['Deceased/Expired',
       'Discharge/Transferred to home under care of organized home health serv',
       'Discharged to home or self-care (routine discharge)',
       'Discharged/Transferred to Long Term Care Hospital',
       'Discharged/Transferred to Skilled Nursing Facility',
       'Discharged/Transferred to a short-term general hospital for inpatient',
       'Discharged/Transferred to another type of institution not defined else',
       'Discharged/Transferred to court/law enforcement',
       'Discharged/Transferred to inpatient rehab or designated unit',
       'Discharged/transferred to a psychiatric hospital or psychiatric distin',
       'Left against medical advice or discontinued care',
       'Not Applicable BIU 1'], dtype=object)